In [5]:
import re
import glob
import re
import string
import nltk
from markovify.chain import Chain
import markovify.text
#import spacy
import markovify
import markovify.text

#nlp = spacy.load("en_core_web_sm")


ModuleNotFoundError: No module named 'nltk'

In [6]:
'''
This part of the code is responsible to combine all different poems of a certain theme into one common file, so that
it can be later use to feed our model.
'''

def glob_re(pattern, strings):
    return filter(re.compile(pattern).match, strings)

nature_themes = "(nature|spring|water|weather|rose)"
depression_themes = "(suicide|sorrow)"

res = [f for f in glob.glob('./sample corpus/topics/**/*.txt') if re.search( depression_themes + '*.txt$', f)]

    
with open('combined.txt', 'w',encoding="utf-8") as outfile:
    for fname in res:
        with open(fname,encoding="utf-8") as infile:
            for line in infile:
                outfile.write(line)

In [7]:

'''
This function calculates the number of syllables present in a line. This function was copied based on english rules and
it can be find in the following website: https://eayd.in/?p=232
'''
def sylco(word) :

    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

In [8]:
'''
Utility function for text cleaning
'''
def text_cleaner(text):
  text = re.sub(r'--', ' ', text)
  text = re.sub('[\[].*?[\]]', '', text)
  text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b','', text)
  text = ' '.join(text.split())
  return text

In [9]:
'''
Writes each line of the compiled poems in a reversed-word order so that we
are able to create a reverse markov model
'''
with open('./combined.txt',encoding="utf-8") as f:
    with open("./result.txt", "w",encoding="utf-8") as output:
        lines = f.readlines()
        for sentence in lines:
            sentence = sentence[:-1]
            sentence = text_cleaner(sentence)
            words = sentence.split(' ') 

            for i in range(len(words)):
                words[i] = words[i].lower().strip(".:;?!,")
                words[i].translate(str.maketrans('', '', string.punctuation))

            reverse_sentence = ' '.join(reversed(words)) + "\n"
            output.write(reverse_sentence)

In [10]:
'''
Using nlk, get words that rhyme with the input*
'''
def get_rhyming_words(input, order):
    pronunDict = dict(nltk.corpus.cmudict.entries())
    rhyme_list = []

    inputSyllables = pronunDict.get(input)

    if inputSyllables:
        rhyme_list += [word for word, pronun in pronunDict.items() if pronun[-order:] == inputSyllables[-order:] and word != input]
        return rhyme_list


In [11]:

DEFAULT_TRIES = 10000 # constant of the number of tries to do to find a type of phrase we are looking for

'''
 Extending the class NewLineText (since we consider that each line should be considered independently as
 we found that pontuaction is absent quite frequently)
'''
class rhymeText(markovify.NewlineText):

    '''
    # if we want to use spacy (takes way longer):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence
    '''
    
    '''
        Constructor adapted from the documentation and source code
    '''
    def __init__(self, input_text, state_size=1, chain=None, parsed_sentences=None, retain_original=True):


        super().__init__(input_text)
        can_make_sentences = parsed_sentences is not None or input_text is not None
        self.retain_original = retain_original and can_make_sentences
        self.state_size = state_size
        self.rhyme_word = ""

        if self.retain_original:
            self.parsed_sentences = list(self.generate_corpus(input_text))

            self.rejoined_text = self.sentence_join(map(self.word_join, self.parsed_sentences))

            self.chain = chain or Chain(self.parsed_sentences, state_size)
        else:
            if not chain:
                parsed = parsed_sentences or self.generate_corpus(input_text)
            self.chain = chain or Chain(parsed, state_size)


    '''
        Tries to find a number of rhyme_words in the chain model, also
        adapted from the documentation and source code
    '''
    def choose_word(self, rhyme_word, matchedWord=None, number=1):
        total = set([]) # set because we do not want repeated ones

        rhyme_list = get_rhyming_words(rhyme_word, 2)

        if rhyme_list is None or len(rhyme_list) == 0 or rhyme_list[0] == None:
            return None


        for word in rhyme_list:
            if len(total) >= number:
                break
            formatted_word = (word,)
            formatted_word_ponctuation = (word + '.',) # some may have this in the end

            if formatted_word in self.chain.model:
                matchedWord = formatted_word
                total.add(matchedWord)

            elif formatted_word_ponctuation in self.chain.model:
                matchedWord = formatted_word_ponctuation
                total.add(matchedWord)

        
        if len(total) < number:
            return None
        return list(total)


    '''
        Generates a sentence with a word that rhymes with rhyme_word and that as a number of syllables
        between min_syllables and max_syllables. The number if the number of sentences it generates
    '''
    def make_short_sentence(self, max_syllables, rhyme_word='', min_syllables=10, number=1 ):
        tries =  DEFAULT_TRIES

        init_state = self.choose_word(rhyme_word, number = number) #4) <===== check this out better
        if init_state == None:
            return None
        
        values = []
        for x in init_state:
            for _ in range(tries):
                sentence = self.make_sentence(init_state=x)
                if sentence and sylco(sentence) <= max_syllables and sylco(sentence) >= min_syllables:
                    values.append(sentence)
                    break
        
        return values

In [12]:

'''
Formats a phrase by putting a uppercase letter in the begining
of the verse
'''
def verse_to_uppercase(sentence):
    return sentence[0].upper() + sentence[1:]



'''
 Since the sentence was generated using the reverse markov chain,
 it reverses the verse and also formats it
'''
def format_reverse_verse(sentence):
    rev = ' '.join(sentence.split()[::-1])
    rhyme = (' ').join(rev.split()[1:])
    return verse_to_uppercase(rhyme)



'''
    Builds a normal markov chain model (using the file with combined poems) 
    and a reverse model (that uses as the input the file that has the verses
    reversed)
'''
def build_markov_model(forward_file, reverse_file):

    with open(forward_file,encoding="utf-8") as f:
        forward_text = f.read()

    with open(reverse_file,encoding="utf-8") as r:
        reverse_text = r.read()

    forward_model = markovify.NewlineText(forward_text)
    reverse_model = rhymeText(reverse_text)

    return (forward_model, reverse_model)


'''
    Gets <size> verses that rhyme with each other
'''       
def build_rhymes(forward_model, reverse_model, min_syllables = 10, max_syllables=50, size = 1):

    A_list = []
    while len(A_list) != size:
        A_list.clear()

        while True:
            A = forward_model.make_short_sentence(40, min_syllables=min_syllables) # A
            if A != None:
                break
        
        rhyme_word = A.split()[-1]
        A_list = []
        X = reverse_model.make_short_sentence(max_syllables, rhyme_word, min_syllables=min_syllables, number=size) #3
        if X != None and len(X) > 0 and X[0] != None:
            A_list = X

    result = [verse_to_uppercase(A)]
    for x in A_list:
        result.append(format_reverse_verse(x))

    return result

'''
    Generates a <size> number of verses that start with a
    certain letter
'''
def build_letter_verse(forward_model, reverse_model, size, letter=None):

    A_list = []
    while len(A_list) != size:

        while True:
            A = forward_model.make_short_sentence(55) # A
            if A != None and A.startswith(letter):
                break
        
        A_list.append(verse_to_uppercase(A))


   
    return A_list


In [13]:
'''
    Builds a sonnet based on the following metric:
    ABBA ABBA CDE CDE
'''
def sonnet(): # ABBA ABBA CDE CDE
    forward_model, reverse_model = build_markov_model("./combined.txt", "./result.txt")

    A = build_rhymes(forward_model, reverse_model, size=4)
    B = build_rhymes(forward_model, reverse_model, size=4)
    C = build_rhymes(forward_model, reverse_model, size=2)
    D = build_rhymes(forward_model, reverse_model, size=2)
    E = build_rhymes(forward_model, reverse_model, size=2)


    ans =  [
        A[0],
        B[0],
        B[1],
        A[1],
        "\n",
        A[2],
        B[2],
        B[3],
        A[3],
        "\n",
        C[0],
        D[0],
        E[0],
        "\n",
        C[1],
        D[1],
        E[1],

    ]

    return '\n'.join(ans)


    

In [15]:
ans = sonnet()
print(ans)

Straight began to draw correct inference
Into the business career
To me of dull till the twilight travel and heart was seen the water beer
Gallons of something nothing to daylight of the absence


Beautiful through the quiet corner of work in one's self acceptance
Spiritual prosperity & they start to the world's hunks of the living and we'll give you get more austere
Don't think not anymore what does appear
Wonder how wide treasure and in the abundance


Now, tell me your sword and squire
And, deep in red and I am able to fly
Of the test is


Of the rose is designed to admire
It’s my world world world i wish them that bound you will hold you apply
Two of his substance abuses


In [16]:
'''
    Builds an acrostic: this type of poem does not need to rhyme. The interesting
    thing about it is that combining the first letter of each verse it should create
    a word
'''
def acrostic(word):
    forward_model, reverse_model = build_markov_model("./combined.txt", "./result.txt")

    ans = []
    for s in word:
        ans = ans + build_letter_verse(forward_model, reverse_model, size=1 , letter=s)

    return '\n'.join(ans)


    

In [17]:
ans = acrostic('rose')
print(ans)

Rose to His feet.
Or am I within a day
So often I taste your infantine toes
Even when I can't remember if she makes it possible


In [18]:
'''
    Builds a limerick based only on the structure it has (not based on its typical theme)
    The poem structure is AABBA and the A verses should be longer than the B verses
'''
def limerick(): # AABBA
    forward_model, reverse_model = build_markov_model("./combined.txt", "./result.txt")

    A = build_rhymes(forward_model, reverse_model, min_syllables= 25, max_syllables=30, size=4)
    B = build_rhymes(forward_model, reverse_model, min_syllables= 10, max_syllables= 15, size=2)

    ans =  [
        A[1],
        A[2],
        B[0],
        B[1],
        A[3]
    ]

    return '\n'.join(ans)



In [19]:
print(limerick())

Pulls out of the coming to the sea gods kept feeling the medicines administered
Say crack on the gentle ripples as you bring back the stage of horn and flowed like the water was acquired
But i need sleep
Picnic by pomp and it just the fallen asleep
Never actually fighting of kindness and the next year be short of love and i'll be admired
